# EIA Demand data via API

In [1]:
import pandas as pd
# import numpy as np
import requests
# import time
import datetime
import json
# from pprint import pprint
# from pandas.io.json import json_normalize

# Import API key
from api_keys import api_key

In [2]:
# API CALL TO USE
# http://api.eia.gov/series/?api_key=YOUR_API_KEY_HERE&series_id=EBA.CISO-ALL.D.HL

url = f'https://api.eia.gov/series/?api_key={api_key}&series_id=EBA.CISO-ALL.D.HL'
response = requests.get(url)

response = requests.get(url)

In [3]:
response_json = response.json()
# print(response_json)

In [4]:
num_resp=len(response_json['series'][0]['data'])
print(num_resp)
print(response_json['series'][0]['data'][0])
print(response_json['series'][0]['data'][-1])

39105
['20191218T09-08', 27494]
['20150701T01-07', 31486]


In [5]:
dict = []
for response in range(num_resp):
    dict.append(response_json['series'][0]['data'][response])

In [6]:
df = pd.DataFrame(dict)
print(df.head())
print(df.tail())

                0        1
0  20191218T09-08  27494.0
1  20191218T08-08  27642.0
2  20191218T07-08  26404.0
3  20191218T06-08  23779.0
4  20191218T05-08  22065.0
                    0        1
39100  20150701T05-07  25661.0
39101  20150701T04-07  26388.0
39102  20150701T03-07  27416.0
39103  20150701T02-07  28989.0
39104  20150701T01-07  31486.0


In [7]:
demand_df = df.rename(columns={0:'timestamp', 1:'Mwh'})

In [8]:
demand_df.head()

,timestamp,Mwh
0,20191218T09-08,27494.0
1,20191218T08-08,27642.0
2,20191218T07-08,26404.0
3,20191218T06-08,23779.0
4,20191218T05-08,22065.0
